In [1]:
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

In [2]:
train_df = pd.read_csv(r'C:\Users\One\Git\ml-ds\as2\flight_delays_train.csv.zip', compression = 'zip')
test_df = pd.read_csv(r'C:\Users\One\Git\ml-ds\as2\flight_delays_test.csv.zip', compression = 'zip')

In [3]:
train_split = train_df.shape[0]
full_df = pd.concat((train_df, test_df))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [4]:
full_df['Duration'] = round(full_df['Distance']/7).astype(np.int32)

In [5]:
#full_df['Duration'] = np.log(full_df['Duration']).astype(np.int32)

In [6]:
full_df['Arrival'] = ((full_df['DepTime'] + full_df['Duration']) // 100)
full_df.loc[full_df['Arrival'] >=24, 'Arrival'] = 1

In [7]:
full_df['LogDistance'] = np.log(full_df['Distance']).astype(np.int32)

In [8]:
#судя по важности не понадобится
#full_df['LogDuration'] = (np.log(full_df['Duration'])**(-2)).astype(np.int32)

In [9]:
#идея
#1 0.7855373409298098
#full_df['Avaiting'] = (full_df['Duration'] / np.log(full_df['Duration'])**(-1)).astype(np.int32)

In [10]:
#время ожидания по отношению к рейсу с более похожим временем
#2 0.7850657026779955
full_df['Avaiting2'] = (full_df['Duration'] / np.log(full_df['Duration'])).astype(np.int32)

In [11]:
#количество маршрутов рейсов у авиакомпаний
#судя по важности не понадобится
full_df['Carrier_trace'] = np.where(full_df['UniqueCarrier'].isin((0, 1, 3, 4, 5,6,7,10,12)), 'high', 
                                        np.where(full_df['UniqueCarrier'].isin((2,8,9,11,14,18,19,20)), 'med', 'low'))

In [12]:
#количество рейсов у аэродрома посадки
Dest_int = train_df.groupby('Dest').count()
df = Dest_int.sort_values(by = ['Month'], ascending = [False])

In [13]:
df['Dest_intense'] = np.where(df['Month'] >=5000, 'a', 
                        (np.where(df['Month'] >=4000, 'b', 
                            (np.where((df['Month'] >= 3000), 'c', 
                                     (np.where((df['Month'] >= 2000), 'd', 
                                               (np.where((df['Month'] >= 1500), 'e', 
                                                        (np.where((df['Month'] >= 1000), 'f', 
                                                                 (np.where((df['Month'] >= 500), 'g', 'h')))))))))))))

In [14]:
a = list(df[df['Dest_intense'] == 'a'].index)
b = list(df[df['Dest_intense'] == 'b'].index)
c = list(df[df['Dest_intense'] == 'c'].index)
d = list(df[df['Dest_intense'] == 'd'].index)
e = list(df[df['Dest_intense'] == 'e'].index)
f = list(df[df['Dest_intense'] == 'f'].index)
g = list(df[df['Dest_intense'] == 'g'].index)
h = list(df[df['Dest_intense'] == 'h'].index)

In [15]:
full_df['Dest_intense'] = np.where(full_df['Dest'].isin(a), 'a',
                                   (np.where(full_df['Dest'].isin(b), 'b',
                                             (np.where(full_df['Dest'].isin(c), 'c',
                                                      (np.where(full_df['Dest'].isin(d), 'd',
                                                               (np.where(full_df['Dest'].isin(e), 'e',
                                                                        (np.where(full_df['Dest'].isin(f), 'f',
                                                                                 (np.where(full_df['Dest'].isin(g), 'g', 'h')))))))))))))

In [16]:
#количество рейсоы у аэродрома вылета
Origin_int = train_df.groupby('Origin').count()
df2 = Origin_int.sort_values(by = ['Month'], ascending = [False])

In [17]:
df2['Origin_intense'] = np.where(df2['Month'] >=5000, 'a2', 
                        (np.where(df2['Month'] >=4000, 'b2', 
                            (np.where((df2['Month'] >= 3000), 'c2', 
                                     (np.where((df2['Month'] >= 2000), 'd2', 
                                               (np.where((df2['Month'] >= 1500), 'e2', 
                                                        (np.where((df2['Month'] >= 1000), 'f2', 
                                                                 (np.where((df2['Month'] >= 500), 'g2', 'h2')))))))))))))

In [18]:
a2 = list(df2[df2['Origin_intense'] == 'a2'].index)
b2 = list(df2[df2['Origin_intense'] == 'b2'].index)
c2 = list(df2[df2['Origin_intense'] == 'c2'].index)
d2 = list(df2[df2['Origin_intense'] == 'd2'].index)
e2 = list(df2[df2['Origin_intense'] == 'e2'].index)
f2 = list(df2[df2['Origin_intense'] == 'f2'].index)
g2 = list(df2[df2['Origin_intense'] == 'g2'].index)
h2 = list(df2[df2['Origin_intense'] == 'h2'].index)

In [19]:
full_df['Origin_intense'] = np.where(full_df['Origin'].isin(a2), 'a2',
                                   (np.where(full_df['Origin'].isin(b2), 'b2',
                                             (np.where(full_df['Origin'].isin(c2), 'c2',
                                                      (np.where(full_df['Origin'].isin(d2), 'd2',
                                                               (np.where(full_df['Origin'].isin(e2), 'e2',
                                                                        (np.where(full_df['Origin'].isin(f2), 'f2',
                                                                                 (np.where(full_df['Origin'].isin(g2), 'g2', 'h2')))))))))))))

In [20]:
full_df['Intense'] = full_df['Origin_intense'] + '-->' + full_df['Dest_intense']

In [21]:
df['Join'] = df.index
df2['Join'] = df2.index

In [22]:
common_intense = pd.merge(df[[ 'Month', 'Join']], df2[['DayofMonth','Join']], how = 'outer')

In [23]:
common_intense['Intensiveness'] = np.where(common_intense['Month'].isna(), 0, (common_intense['DayofMonth'] / common_intense['Month']))

In [24]:
common_intense['Intensiveness'] = common_intense['Intensiveness'].fillna(0).astype(object)

In [25]:
new_full_df = full_df.merge(common_intense[['Intensiveness', 'Join']], left_on='Dest', right_on='Join', how = 'outer')

In [27]:
new_full_df['flight'] = new_full_df['Origin'] + '-->' + new_full_df['Dest']

# Hour and minute
new_full_df['hour'] = new_full_df['DepTime'] // 100
new_full_df.loc[new_full_df['hour'] == 24, 'hour'] = 0
new_full_df.loc[new_full_df['hour'] == 25, 'hour'] = 1
new_full_df['minute'] = new_full_df['DepTime'] % 100

# Daytime
new_full_df['daytime'] = pd.cut(new_full_df['hour'], bins=[0, 6, 12, 18, 23], include_lowest=True).astype(object)

for col in ['Origin', 'UniqueCarrier', 'daytime']:
    new_full_df[col] = pd.factorize(new_full_df[col])[0]

In [28]:
for col in ['Origin', 'Dest', 'UniqueCarrier', 'daytime']:
    new_full_df[col] = pd.factorize(new_full_df[col])[0]
#test_df['flight'] = test_df['Origin'] + '-->' + test_df['Dest']
#full_df['DayOfWeek'] = full_df['DayOfWeek'].apply(lambda x: x.replace('c-',''))
#full_df['Month'] = full_df['Month'].apply(lambda x: x.replace('c-',''))
#full_df['DayofMonth'] = full_df['DayofMonth'].apply(lambda x: x.replace('c-',''))

In [29]:
bin = [0, 1, 2, 3, 4]
new_full_df['DistanceBin'] = pd.cut(new_full_df['Distance'], bins=[0,100,300,800,1500,3000], labels = bin, include_lowest = True)
new_full_df['DistanceBin'].fillna(0, inplace = True)

In [30]:
new_full_df['summer'] = (new_full_df['Month'].isin(['c-6', 'c-7', 'c-8'])).astype(np.int32)
new_full_df['autumn'] = (new_full_df['Month'].isin(['c-9', 'c-10', 'c-11'])).astype(np.int32)
new_full_df['winter'] = (new_full_df['Month'].isin(['c-12', 'c-1', 'c-2'])).astype(np.int32)
new_full_df['spring'] = (new_full_df['Month'].isin(['c-3', 'c-4', 'c-5'])).astype(np.int32)

In [31]:
#поменять категориальные фичи тут
train_df, test_df = new_full_df.iloc[:train_split], new_full_df.iloc[train_split:]
categ_feat_idx = np.where(train_df.drop(['dep_delayed_15min', 'Join', 'Carrier_trace', 'Intensiveness'], axis=1).dtypes)[0]
new_full_df['Intensiveness'].fillna(0, inplace = True)
new_full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 199999
Data columns (total 28 columns):
DayOfWeek            200000 non-null object
DayofMonth           200000 non-null object
DepTime              200000 non-null int64
Dest                 200000 non-null int64
Distance             200000 non-null int64
Month                200000 non-null object
Origin               200000 non-null int64
UniqueCarrier        200000 non-null int64
dep_delayed_15min    100000 non-null object
Duration             200000 non-null int32
Arrival              200000 non-null int64
LogDistance          200000 non-null int32
Avaiting2            200000 non-null int32
Carrier_trace        200000 non-null object
Dest_intense         200000 non-null object
Origin_intense       200000 non-null object
Intense              200000 non-null object
Intensiveness        200000 non-null float64
Join                 199917 non-null object
flight               200000 non-null object
hour             

In [33]:
# Categorical columns
cat_cols = ['Month', 'DayofMonth', 'DayOfWeek', 'Origin', 'Dest', 'UniqueCarrier', 
            'hour', 'summer', 'autumn', 'winter', 'spring', 'daytime', 'flight']

# Converting categorical columns to type 'category' as required by LGBM
for c in cat_cols:
    full_df[c] = new_full_df[c].astype('category')

In [34]:
#поменять категориальные фичи тут
X_train = train_df.drop(['dep_delayed_15min', 'Join', 'Carrier_trace', 'Intensiveness'], axis=1).values
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test_df.drop(['dep_delayed_15min', 'Join', 'Carrier_trace', 'Intensiveness'], axis=1).values

In [35]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, 
                                                                test_size=0.3, 
                                                                random_state=18)

In [36]:
ctb = CatBoostClassifier(random_seed=18, silent=True, depth=9, boosting_type = 'Plain', iterations = 2000, 
                         l2_leaf_reg=3, grow_policy = 'SymmetricTree', leaf_estimation_method = 'Gradient')

In [37]:
ctb.fit(X_train_part, y_train_part, cat_features=categ_feat_idx);

In [38]:
ctb_valid_pred = ctb.predict_proba(X_valid)[:, 1]

In [42]:
roc_auc_score(y_valid, ctb_valid_pred)

ValueError: continuous format is not supported

In [43]:
ctb.feature_importances_

array([18.0744339 , 18.46941088,  2.18838766,  2.82682561,  1.93478581,
       18.34235001,  2.29028565,  6.9836784 ,  1.52393562,  4.13658831,
        1.09329842,  1.50820559,  1.40423694,  1.17722519,  1.35637911,
        2.38565087,  3.95808603,  2.75040858,  5.83623476,  1.02138728,
        0.24497326,  0.11461071,  0.24715538,  0.13146604])

In [44]:
ctb.fit(X_train, y_train, cat_features=categ_feat_idx);

In [45]:
ctb_test_pred = ctb.predict_proba(X_test)[:, 1]

In [46]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    sample_sub = pd.read_csv(r'C:\Users\One\Git\ml-ds\as2\sample_submission.csv.zip', compression = 'zip', 
                             index_col='id')
    sample_sub['dep_delayed_15min'] = ctb_test_pred
    sample_sub.to_csv('submit_31.csv')